In [ ]:
# torch with cuda
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from ultralytics import YOLO
import torch
import os

# Load a model
# model = YOLO("best.onnx")
yolo_model = YOLO("yolo11n.pt")

path = os.path.join(os.path.abspath("."),"License Plate Recognition.v6i.yolov11.zip")

# # Train the model
# train_results = model.train(
#     data=path,  # path to dataset YAML
#     epochs=100,  # number of training epochs
#     # workers=100,
#     # patience=100,
#     imgsz=640,  # training image size
#     device="cuda" if torch.cuda.is_available() else "cpu",  # device to run on, i.e. device=0 or device=0,1,2,3 or device=cpu
# )

# # Evaluate model performance on the validation set
# metrics = model.val()

# # Export the model to ONNX format
# path = model.export(format="onnx")  # return path to exported model

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import cv2
from matplotlib import pyplot as plt
from safe_video import ObjectDetection

rec = ObjectDetection()
image = cv2.imread("example.jpg")
image = image[:, :, ::-1]
res = rec.chained_detection(image, "car", "License_Plate")
blured = rec.blur_image(image, rec.result, "License_Plate")
plt.imshow(blured)
plt.show()
plt.imshow(rec.result.plot())
plt.show()
rec.analyze(image, ["traffic light", "car", "License_Plate", "barbie"])
plt.imshow(rec.result.plot())
plt.show()

In [ ]:
print(rec.get_classes())
# print(rec.result)